In [41]:
!pip install tensorflow


In [42]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


In [43]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import os
os.chdir('/content/drive/My Drive/ML-class/project/dataset')


In [45]:
train_dir = "/content/drive/My Drive/ML-class/project/dataset/train"
test_dir = "/content/drive/My Drive/ML-class/project/dataset/test"

In [46]:
train_skin_types=os.listdir(train_dir)
train_skin_types


['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [47]:
number_of_skin_types = len(train_skin_types)

print("Number of skin types in train folder:", number_of_skin_types)


Number of skin types in train folder: 23


In [48]:
test_skin_types=os.listdir(test_dir)
test_skin_types


['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [49]:
number_of_skin_types = len(test_skin_types)

print("Number of skin types in test folder:", number_of_skin_types)


Number of skin types in test folder: 23


In [50]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

# Define categories
categories = {
    'train': ['Acne and Rosacea Photos',
              'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
              'Atopic Dermatitis Photos',
              'Bullous Disease Photos',
              'Cellulitis Impetigo and other Bacterial Infections',
              'Eczema Photos',
              'Exanthems and Drug Eruptions',
              'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
              ],
    'test': ['Acne and Rosacea Photos',
              'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
             'Atopic Dermatitis Photos',
              'Bullous Disease Photos',
              'Cellulitis Impetigo and other Bacterial Infections',
             'Eczema Photos',
            'Exanthems and Drug Eruptions',
             'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
             ]
}

# Define paths
base_path = "/content/drive/My Drive/ML-class/project/dataset"
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

# Function to load data
def load_data(path, categories):
    data = {"image_path": [], "target": []}
    for idx, category in enumerate(categories):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):  # Check if the path is a directory
            for img_name in os.listdir(category_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):  # Check for image files
                    img_path = os.path.join(category_path, img_name)
                    data["image_path"].append(img_path)
                    data["target"].append(idx)
    return pd.DataFrame(data)

# Load train and test data
train_df = load_data(train_path, categories['train'])
test_df = load_data(test_path, categories['test'])

# Function to preprocess and load images
def load_images(image_paths):
    images = []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (180, 180))
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return images

# Take a sample of the data for quick testing
sample_train_df = train_df.sample(frac=0.3, random_state=42)
sample_test_df = test_df.sample(frac=0.1, random_state=42)

# Load images
train_images = load_images(sample_train_df['image_path'])
test_images = load_images(sample_test_df['image_path'])

# # Display example images
# example_imgs = random.sample(train_images, 6)
# fig, axes = plt.subplots(2, 3, figsize=(10, 8))
# for ax, img in zip(axes.ravel(), example_imgs):
#     ax.imshow(img)
#     ax.axis('off')
# plt.show()


In [51]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

# Convert lists of images to numpy arrays
train_images = np.array(train_images)
test_images = np.array(test_images)

# Convert lists of labels to numpy arrays
train_labels = np.array(sample_train_df['target'])
test_labels = np.array(sample_test_df['target'])

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Data Augmentation
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
])

# Create MobileNetV2 Model
base_model = tf.keras.applications.MobileNetV2(input_shape=(180, 180, 3), include_top=False, weights='imagenet')
base_model.trainable = False

number_of_classes = 10  # Update this based on your dataset
model = Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(number_of_classes, activation='softmax')
])

# # Compile the Model
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [54]:
#Train the Model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=32)



Epoch 1/20
46/46 [==============================] - 16s 146ms/step - loss: 2.4101 - accuracy: 0.2205 - val_loss: 1.9400 - val_accuracy: 0.3599
Epoch 2/20
46/46 [==============================] - 3s 62ms/step - loss: 2.0419 - accuracy: 0.3201 - val_loss: 1.8637 - val_accuracy: 0.3764
Epoch 3/20
46/46 [==============================] - 3s 62ms/step - loss: 1.8367 - accuracy: 0.3984 - val_loss: 1.7352 - val_accuracy: 0.3956
Epoch 4/20
46/46 [==============================] - 3s 60ms/step - loss: 1.7467 - accuracy: 0.3977 - val_loss: 1.7217 - val_accuracy: 0.4176
Epoch 5/20
46/46 [==============================] - 2s 52ms/step - loss: 1.7177 - accuracy: 0.4162 - val_loss: 1.6999 - val_accuracy: 0.4478
Epoch 6/20
46/46 [==============================] - 2s 46ms/step - loss: 1.6707 - accuracy: 0.4306 - val_loss: 1.6793 - val_accuracy: 0.4643
Epoch 7/20
46/46 [==============================] - 2s 52ms/step - loss: 1.6045 - accuracy: 0.4663 - val_loss: 1.6862 - val_accuracy: 0.4423
Epoch 8/20


In [55]:
# Evaluate the Model
loss, accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy: ", accuracy)

6/6 [==============================] - 1s 119ms/step - loss: 1.7988 - accuracy: 0.4074
Test accuracy:  0.40740740299224854


In [56]:
def predict_skin_disease(image_path):
    # Define list of class names
    class_names = ['Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
                   'Acne and Rosacea Photos','Atopic Dermatitis Photos',
                    'Bullous Disease Photos',
                    'Cellulitis Impetigo and other Bacterial Infections',
             'Eczema Photos',
            'Exanthems and Drug Eruptions',
                   'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
                  ]

    # Load and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (180, 180))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # Make prediction on preprocessed image
    pred = model.predict(img)
    predicted_class_index = np.argmax(pred)
    predicted_class_name = class_names[predicted_class_index]

    return predicted_class_name, pred

img_path = test_dir+'/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-cheilitis-sq-cell-lip-6.jpg'
predicted_class, pred = predict_skin_disease(img_path)
print(predicted_class)
print("Prediction probabilities:", pred)



1/1 [==============================] - 1s 1s/step
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
Prediction probabilities: [[6.3543963e-01 1.1390808e-01 2.0755485e-03 1.6967784e-01 3.9399615e-03
  4.0156923e-02 9.6932193e-04 2.1530569e-03 2.1220968e-04 3.1467479e-02]]


In [58]:
!pip install scikit-image


**SVM**

In [59]:
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Function to extract HOG features from a list of images
def extract_hog_features(images):
    hog_features = []
    for image in images:
        features = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)
        hog_features.append(features)
    return hog_features

# Extract HOG features
train_hog_features = extract_hog_features(train_images)
test_hog_features = extract_hog_features(test_images)

# Normalize the features - important for SVM
scaler = StandardScaler()
train_hog_features = scaler.fit_transform(train_hog_features)
test_hog_features = scaler.transform(test_hog_features)

# Reduce the dimensionality of the features
pca = PCA(n_components=0.9) # Retain 90% of the variance
train_hog_features_pca = pca.fit_transform(train_hog_features)
test_hog_features_pca = pca.transform(test_hog_features)

# Train an SVM model
svm_model = svm.SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(train_hog_features_pca, train_labels)

# Predict on the test set
svm_predictions = svm_model.predict(test_hog_features_pca)

# Evaluate the model
print("Classification report for SVM classifier:")
print(classification_report(test_labels, svm_predictions))
print("Test accuracy: ", accuracy_score(test_labels, svm_predictions))


<ipython-input-59-2d7f6c5accad>:11: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  features = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)


Classification report for SVM classifier:
              precision    recall  f1-score   support

           0       0.53      0.53      0.53        38
           1       0.32      0.48      0.38        31
           2       0.60      0.25      0.35        12
           3       0.33      0.33      0.33         9
           4       0.00      0.00      0.00         4
           5       0.36      0.52      0.42        27
           6       0.38      0.18      0.24        17
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         7
           9       0.20      0.15      0.17        13

    accuracy                           0.37       162
   macro avg       0.27      0.24      0.24       162
weighted avg       0.36      0.37      0.35       162

Test accuracy:  0.37037037037037035


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
def predict_image_class(image_path, scaler, pca, svm_model, class_names):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (180, 180)) # Resize to the size expected by the model
    img = img / 255.0  # Normalize pixel values to [0, 1]

    # Extract HOG features
    features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)

    # Normalize the features using the same scaler used during training
    features = scaler.transform([features])

    # Reduce the dimensionality using PCA used during training
    features_pca = pca.transform(features)

    # Predict using the SVM model
    prediction = svm_model.predict(features_pca)
    predicted_class = class_names[prediction[0]]

    return predicted_class

# Assuming class_names is a list of string labels for each class, for example:
class_names = ['Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
                   'Acne and Rosacea Photos','Atopic Dermatitis Photos',
                    'Bullous Disease Photos',
                    'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos',
              'Exanthems and Drug Eruptions',
               'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
              ]

# Example usage:
img_path = test_dir+'/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-cheilitis-sq-cell-lip-6.jpg'
predicted_class = predict_image_class(img_path, scaler, pca, svm_model, class_names)
print(f'The predicted class is: {predicted_class}')


The predicted class is: Acne and Rosacea Photos


<ipython-input-60-9e02d4b977ee>:9: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)


**Reference:**

1. https://www.kaggle.com/code/ishaanchandak301/skin-detection-acc-98